In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
sys.path.append('..')
from tqdm import tqdm
import pandas as pd
import warnings
from src.lesion import perform_lesion_experiment, do_lesion_hypo_tests
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path

Using TensorFlow backend.


In [2]:
models = [('mnist', 'MNIST'),
          ('fashion', 'FASHION'), ('poly', 'POLY')
          # ('cifar10', 'CIFAR10'),
          # ('mnist', 'MNIST+DROPOUT'), ('cifar10', 'CIFAR10+DROPOUT'),
          # ('fashion', 'FASHION+DROPOUT'),
          ]

n_clust = 12
n_shuffles = 20
n_workers = 5
n_reps = 5
is_unpruned = False

In [3]:
all_results = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    for dataset_name, tag in tqdm(models):

        paths = get_model_path(tag, filter_='all')[-n_reps:]
        for path in paths:

            (true_results,
             all_random_results,
             metadata,
             evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                     path, n_clusters=n_clust,
                                                     n_shuffles=n_shuffles,
                                                     unpruned=is_unpruned)
            hypo_results = do_lesion_hypo_tests(evaluation, true_results, all_random_results)
            chi2_p_means = hypo_results['chi2_p_means']
            chi2_p_ranges = hypo_results['chi2_p_ranges']
            combined_p_means = hypo_results['combined_p_means']
            combined_p_ranges = hypo_results['combined_p_ranges']
            effect_means = hypo_results['effect_factor_means']
            effect_ranges = hypo_results['effect_factor_range']

            model_results = {'is_unpruned': is_unpruned, 'model_tag': tag,
                             'dataset': dataset_name, 'chi2_p_means': chi2_p_means,
                             'chi2_p_ranges': chi2_p_ranges, 'combined_p_means': combined_p_means,
                             'effect_ranges': effect_ranges, 'effect_means': effect_means,
                             'combined_p_ranges': combined_p_ranges}
            all_results.append(pd.Series(model_results))

result_df = pd.DataFrame(all_results)
savepath = '../results/lesion_results_mlp.csv'
result_df.to_csv(savepath)
result_df


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [30:17<1:00:35, 1817.97s/it]

 67%|██████▋   | 2/3 [1:04:01<31:19, 1879.75s/it]

100%|██████████| 3/3 [1:36:04<00:00, 1892.64s/it]

100%|██████████| 3/3 [1:36:04<00:00, 1921.53s/it]

,is_unpruned,model_tag,dataset,chi2_p_means,chi2_p_ranges,combined_p_means,effect_ranges,effect_means,combined_p_ranges
0,False,MNIST,mnist,2.616959e-12,1.972133e-01,3.723754e-07,5.141354e+00,1.054148,7.724322e-01
1,False,MNIST,mnist,4.397937e-14,3.850862e-04,1.341372e-08,9.705171e+00,1.033126,9.846316e-01
2,False,MNIST,mnist,1.467892e-16,4.110047e-03,1.194488e-08,1.009853e+01,1.024116,9.677028e-01
3,False,MNIST,mnist,8.338514e-13,8.775936e-02,5.802777e-08,3.035395e+00,1.023603,5.111085e-01
4,False,MNIST,mnist,1.985740e-17,3.470003e-02,2.653024e-09,1.107928e+01,1.025438,9.762319e-01
5,False,FASHION,fashion,1.759935e-11,8.556220e-09,7.730191e-06,inf,1.202914,8.615767e-01
6,False,FASHION,fashion,3.306690e-11,1.504932e-03,9.302282e-05,7.300478e+11,1.090869,2.198356e-01
7,False,FASHION,fashion,4.910232e-22,3.408331e-02,1.627973e-07,3.978180e+11,1.147517,7.137569e-01
8,False,FASHION,fashion,4.660709e-18,3.776269e-02,5.270228e-06,inf,1.126919,1.453712e-01
9,False,FASHION,fashion,9.311098e-08,2.586911e-03,4.420465e-05,inf,1.153860,5.131659e-01
